In [1]:
# !pip install haversine

In [1]:
import pandas as pd
import numpy as np 
from haversine import haversine
import datetime
import json

In [6]:
PATH = "./server/data/"
ETC = "./server/data/2. 미래기술 챌린지 제공 Data(참고).xlsx"
TERMINAL = "./server/data/HubLocation.xlsx"
LOCATION = "./server/data/시군구_위도_경도.csv"
get_terminal_info = "./server/data/get_terminal.json"
send_terminal_info = "./server/data/send_terminal_result.json"

In [7]:
### 데이터 로드

e_commerce_fc_3 = pd.read_feather(PATH + "fc_3.ftr", columns=["CORP_ID", "BKG_DATE", "BKG_TIME", "SHPR_CD", "ITEM_QTY", "INS_ID", "POST_ZONE", "SHPR_ADDR_1", "SHPR_ADDR_2", "CNEE_ADDR_1", "CNEE_ADDR_2"])
e_commerce_fc_4 = pd.read_feather(PATH + "fc_4.ftr", columns=["CORP_ID", "BKG_DATE", "BKG_TIME", "SHPR_CD", "ITEM_QTY", "INS_ID", "POST_ZONE", "SHPR_ADDR_1", "SHPR_ADDR_2", "CNEE_ADDR_1", "CNEE_ADDR_2"])
e_commerce_fc_5 = pd.read_feather(PATH + "fc_5.ftr", columns=["CORP_ID", "BKG_DATE", "BKG_TIME", "SHPR_CD", "ITEM_QTY", "INS_ID", "POST_ZONE", "SHPR_ADDR_1", "SHPR_ADDR_2", "CNEE_ADDR_1", "CNEE_ADDR_2"])
e_commerce_fc_6 = pd.read_feather(PATH + "fc_6.ftr", columns=["CORP_ID", "BKG_DATE", "BKG_TIME", "SHPR_CD", "ITEM_QTY", "INS_ID", "POST_ZONE", "SHPR_ADDR_1", "SHPR_ADDR_2", "CNEE_ADDR_1", "CNEE_ADDR_2"])

info = pd.read_excel(ETC)

with open(get_terminal_info, 'r') as f:
    get_terminal = json.load(f)

with open(send_terminal_info, 'r') as fb:
    send_terminal_result = json.load(fb)

terminal_info = pd.read_excel(TERMINAL, sheet_name=0)
location_info = pd.read_csv(LOCATION)


e_commerce_fc_data = pd.concat([e_commerce_fc_3, e_commerce_fc_4, e_commerce_fc_5, e_commerce_fc_6])
e_commerce_fc_data.reset_index(inplace=True)
e_commerce_fc_data.drop("index",  axis=1, inplace=True)

In [8]:
### 데이터 컬럼 rename

info_list_eng = info["Unnamed: 1"]
info_list_kr = info["Unnamed: 2"]

info_list_eng = info_list_eng.drop(labels=[12, 20])
info_list_kr = info_list_kr.drop(labels=[12, 20])

column_dict = {}

for eng, kr in zip(info_list_eng, info_list_kr):
    column_dict[eng] = kr


    e_commerce_fc_data.rename(columns = column_dict, inplace=True)
e_commerce_fc_data.rename(columns = {
    "창고코드(KX007: 곤지암, GP001: 군포)" : "창고코드", 
    "고객 주문번호, 해당 테이블 key값, 네이버 채번" : "고객주문번호", 
    "주문유형(7: B2C출고, 8: 정상반출, 9: 불량반출)" : "주문유형" 
}, inplace=True)

e_commerce_fc_data.rename(columns = {
    "CJ대한통운 주문번호(예약번호)" : "주문번호", 
    "입력일자(인터페이스 시간)" : "입력일자", 
    "입력시간(인터페이스시간)" : "입력시간", 
    "SHPR_ADDR_1" : "송화인주소1", 
    "SHPR_ADDR_2" : "송화인주소2", 
    "CNEE_ADDR_1" : "수화인주소1", 
    "CNEE_ADDR_2" : "수화인주소2"
}, inplace=True)

In [9]:
### 데이터 주문 날짜, 주문 시간 중심으로 데이터 index 설정

time_list = []
for i in range(len(e_commerce_fc_data)):
    days = str(e_commerce_fc_data["주문 날짜"][i])
    year = days[:4]
    month = days[4:6]
    day = days[6:]

    length = len(str(e_commerce_fc_data["주문 시간"][i]))
    str_time = str(e_commerce_fc_data["주문 시간"][i])
    if length == 1:
        times = "00000" + str_time
    elif length == 2:
        times = "0000" + str_time
    elif length == 3:
        times = "000" + str_time
    elif length == 4:
        times = "00" + str_time
    elif length == 5:
        times = "0" + str_time
    else:
        times = str_time

    h = times[:2]
    m = times[2:4]
    s = times[4:6]
    time_list.append(datetime.datetime(int(year), int(month), int(day), int(h), int(m), int(s)))

e_commerce_fc_data.index = time_list
e_commerce_fc_data["datetime"] = time_list


In [10]:
### 송화인 터미널, 수화인 터미널 생성
e_commerce_fc_data.dropna(subset=["수화인주소1"], inplace=True)

get_terminal_1 = []

for terminal in e_commerce_fc_data["수화인주소1"] : 
    if terminal != None : 
        try:            
            get_terminal_1.append(get_terminal[terminal]) 
        except:
            get_terminal_1.append("없음")

send_terminal = {}

send_terminal["경기도"] = "경기도"
send_terminal["경기"] = "경기도"

send_terminal_1 = []

for terminal in e_commerce_fc_data["송화인주소1"] : 
    if terminal == "경기도" or terminal == "경기" : 
        send_terminal_1.append(send_terminal[terminal])
    else : 
        send_terminal_1.append(terminal)

send_terminal_2 = e_commerce_fc_data["송화인주소2"]
get_terminal_2 = e_commerce_fc_data["수화인주소2"]


send_spot = []
get_spot = []

for send1, send2, get1, get2 in zip(send_terminal_1, send_terminal_2, get_terminal_1, get_terminal_2) :
    send_result = "%s %s" %(send1, send2)
    get_result = "%s %s" %(get1, get2)
    send_spot.append(send_result)
    get_spot.append(get_result)


e_commerce_fc_data['송화인터미널'] = send_spot
e_commerce_fc_data["수화인터미널"] = get_spot

In [11]:
## 송화인 수화인 주소별 터미널 매치

# 곤지암 Hub
HUB_GONJIAM = terminal_info[terminal_info["Hub"] == "MegaHub곤지암"]
HUB_GONJIAM_LATITUDE = HUB_GONJIAM["Latitude"].iloc[0]
HUB_GONJIAM_LONGITUDE = HUB_GONJIAM["Longitude"].iloc[0]

# 군포 Hub
HUB_GUNPO = terminal_info[terminal_info["Hub"] == "군포Hub"]
HUB_GUNPO_LATITUDE = HUB_GUNPO["Latitude"].iloc[0]
HUB_GUNPO_LONGITUDE = HUB_GUNPO["Longitude"].iloc[0]

# terminal dictionary 생성
terminal_hub_dict = terminal_info.to_dict()["Hub"]
terminal_latitude_dict = terminal_info.to_dict()["Latitude"]
terminal_longitude_dict = terminal_info.to_dict()["Longitude"]

terminal_hub_swap = {k : v for v, k in terminal_hub_dict.items()}

# locaiton dictionary 생성
location_dict = location_info.to_dict()["시군구"]
location_latitude_dict = location_info.to_dict()["위도"]
location_longitude_dict = location_info.to_dict()["경도"]

location_dict_swap = {k : v for v, k in location_dict.items()}

send_latitudes_list = []
send_longitudes_list = []
send_hubs_list = []

count = 0

for send_terminal in e_commerce_fc_data["송화인터미널"] :
    if send_terminal == "경기도 안산시" :
        latitude = location_info[location_info["시군구"] == "경기도 안산시"]["위도"].iloc[0]
        longitude = location_info[location_info["시군구"] == "경기도 안산시"]["경도"].iloc[0]
        hub = "안산시청"
    else :
        if send_terminal not in send_terminal_result.keys():
            latitude = 0
            longitude = 0
            hub = "없음"
        else:
            latitude = terminal_latitude_dict[terminal_hub_swap[send_terminal_result[send_terminal]]]
            longitude = terminal_longitude_dict[terminal_hub_swap[send_terminal_result[send_terminal]]]
            hub = send_terminal_result[send_terminal]



    send_latitudes_list.append(latitude)
    send_longitudes_list.append(longitude)
    send_hubs_list.append(hub)


location_result_dict = {}

for location, hub in zip(terminal_info["Location"], terminal_info["Hub"]) : 
    split_location_info = location.split(" ")
    big_location = split_location_info[0] + " " + split_location_info[1]

    keys = location_result_dict.get(big_location, [])
    keys.append(hub)

    location_result_dict[big_location] = keys

latitudes = []
longitudes = []
location = []

    
for get_terminal in e_commerce_fc_data["수화인터미널"] : 
    terminal = location_result_dict.get(get_terminal, "시청")

    if terminal == "시청" :

        if get_terminal in location_dict_swap.keys():   
            location.append(get_terminal)
            latitudes.append(location_latitude_dict[location_dict_swap[get_terminal]])      
            longitudes.append(location_longitude_dict[location_dict_swap[get_terminal]])  
        elif "세종특별자치시" in get_terminal: 
            location.append("세종Sub")
            latitudes.append(36.544)
            longitudes.append(127.357)
        else :  
            location.append("매치 x")
            latitudes.append(0)
            longitudes.append(0)
    else : 
        location.append(terminal[-1])    
        latitudes.append(terminal_latitude_dict[terminal_hub_swap[terminal[-1]]])
        longitudes.append(terminal_longitude_dict[terminal_hub_swap[terminal[-1]]])


e_commerce_fc_data["SHPR_ADDR_LATITUDE"] =  send_latitudes_list
e_commerce_fc_data["SHPR_ADDR_LONGITUDE"] =  send_longitudes_list
e_commerce_fc_data["SHPR_ADDR_HUBS"] = send_hubs_list

e_commerce_fc_data["CNEE_ADDR_LATITUDE"] =  latitudes
e_commerce_fc_data["CNEE_ADDR_LONGITUDE"] =  longitudes
e_commerce_fc_data["CNEE_ADDR_HUBS"] = location

e_commerce_fc_data = e_commerce_fc_data[(e_commerce_fc_data["SHPR_ADDR_LATITUDE"] != 0) &
                                        (e_commerce_fc_data["SHPR_ADDR_LONGITUDE"] != 0) &
                                        (e_commerce_fc_data["CNEE_ADDR_LATITUDE"] != 0) &
                                        (e_commerce_fc_data["CNEE_ADDR_HUBS"] != 0)
                                        ]

In [12]:
### 거리 계산
location_e_commerce = e_commerce_fc_data[["창고코드",
                                         'SHPR_ADDR_LATITUDE', 
                                         'SHPR_ADDR_LONGITUDE', 
                                         'SHPR_ADDR_HUBS',
                                         'CNEE_ADDR_LATITUDE', 
                                         'CNEE_ADDR_LONGITUDE', 
                                         'CNEE_ADDR_HUBS'
                                         ]]


### 실제 거리 비교 W = 1.5
# distance = ((SHPR_terminal, megaHub) + (megaHub, CNEE_terminal)) * 1.5 

KX007 = (37.384, 127.314)
GP001 = (37.331, 126.938)

distance_list = []
for i in range(len(location_e_commerce)):
    if location_e_commerce["창고코드"][i] == "KX007":
        target = KX007
    else:
        target = GP001
    location_set1 = (location_e_commerce["SHPR_ADDR_LATITUDE"][i], location_e_commerce["SHPR_ADDR_LONGITUDE"][i])
    location_set2 = (location_e_commerce["CNEE_ADDR_LATITUDE"][i], location_e_commerce["CNEE_ADDR_LONGITUDE"][i])   

    distance1 = haversine(location_set1, target)
    distance2 = haversine(target, location_set2)

    if distance1 < 30:
        distance1 = distance1 * 1.5
    elif (distance1 >= 30) and (distance1 < 100):
        distance1 = distance1 * 1.4
    elif (distance1 >= 100) and (distance1 < 200):
        distance1 = distance1 * 1.3
    else:
        distance1 = distance1 * 1.2

    if distance2 < 30:
        distance2 = distance2 * 1.5
    elif (distance2 >= 30) and (distance2 < 100):
        distance2 = distance2 * 1.4
    elif (distance2 >= 100) and (distance2 < 200):
        distance2 = distance2 * 1.3
    else:
        distance2 = distance2 * 1.2

    distance_list.append(distance1 + distance2)

e_commerce_fc_data["distance"] = distance_list

In [ ]:
### 송화인, 수화인 위치 
SHPR_set = set()
CNEE_set = set()
for i in range(len(location_e_commerce)):
    SHPR_set.add((location_e_commerce["SHPR_ADDR_LATITUDE"][i], location_e_commerce["SHPR_ADDR_LONGITUDE"][i]))
    CNEE_set.add((location_e_commerce["CNEE_ADDR_LATITUDE"][i], location_e_commerce["CNEE_ADDR_LONGITUDE"][i]))

shpr_location = pd.DataFrame(SHPR_set, columns = ["latitude", "longitude"])
cnee_location = pd.DataFrame(CNEE_set, columns = ["latitude", "longitude"])    

In [13]:
print("Done !!")

Done !!
